In [ ]:
import json
with open('value_graph_data1.json', 'r', encoding='utf-8') as f:
    data = json.load(f)
    print(data)
    nodes = {}
    edges = []
    nodes_j = data['nodes']
    edges_j = data['edges']
    index = 1
    # print(nodes_j.keys())
    # print(edges_j)
    for key, value in nodes_j.items():
        nodes[key] = {
            "id":index,
            "label":key,
            "value":value,
            "mass":1
        }
        index += 1
    
    for model, model_edges in edges_j.items():
        print(model)
        for edge in model_edges:
            edges.append({
                "from":nodes[edge[0]]["id"],
                "to":nodes[edge[1]]["id"],
                "weight":edge[2],
                "line":edge[3],
                "label": "->".join(edge[0:1]),
                "model": model
            })
            nodes[edge[0]]["mass"] += 1
            nodes[edge[1]]["mass"] += 1

print(nodes)
print(edges)
    


In [ ]:
import math
from pyvis.network import Network
import networkx as nx

def get_circle_nodes(n, r, x_center, y_center):
    nodes = []
    for i in range(n):
        angle = 2 * math.pi * i / n  # 计算每个节点的角度
        x_i = x_center + r * math.cos(angle)  # 计算x坐标
        y_i = y_center + r * math.sin(angle)  # 计算y坐标
        nodes.append((x_i, y_i))
    return nodes

positions = get_circle_nodes(len(nodes), r=600, x_center=250, y_center=250)
print(positions)
model_color = {
    "gemma":"#F4C6E5", # 粉色
    "llama":"#54DEFD", # 蓝色
    "reference":"#B1D076" #绿色
}
model_smooth = {
    "gemma":{"type":"curvedCW", "roundness": 0.2}, 
    "llama":{"type":"curvedCCW", "roundness": 0.2}, 
    "reference":{"type":"curvedCW", "roundness": 0}
}
edge_line = {
    "single-arrow":"to",
    "no-arrow":"",
    "double-arrow": "to,from"
}

#g = nx.MultiGraph()
g = nx.MultiDiGraph()
for node in nodes:
    g.add_node(nodes[node]['id'], label=node, color='#FEAD34', shape='dot', size=nodes[node]['mass']*3,  mass=nodes[node]['mass'])
id = 0
for edge in edges:
    #if edge["model"] != "llama":
    if True:
        g.add_edge(edge['from'], edge['to'], id=id, lebel=edge["label"], smooth=model_smooth[edge["model"]], color=model_color[edge["model"]], arrows=edge_line[edge["line"]], width=2)
    id += 1

net = Network(directed=True, height='1000px', width='50%', bgcolor='white', font_color='#222222',  select_menu=True, filter_menu=True)

net.from_nx(g)
#add legend for each model, align the legends' postions in a row
y = 0
for model in model_color:
    #if edge["model"] != "llama":
    if True:
        net.add_node(model, label=model, color=model_color[model], shape='box', size=3, mass=1, x=0, y=y, physics=False)
        y += 1
    

net.show_buttons(filter_=['physics'])
net.toggle_physics(True)
net.set_edge_smooth('dynamic')
net.show('simple_network.html', notebook=False)
